In [1]:
import pandas as pd
import numpy as np
import pyomo.environ as pyo
from numpy.random import normal

In [2]:
#  weight parameters are uncertain with normal distribution and known mean, standard deviation
df = pd.read_csv('knapsack_dataset.csv')  
print(df)

     name  value  weight_mean  weight_std
0    gold    100          4.0         0.1
1  silver     90          2.8         0.4
2  bronze     70          2.2         0.3


In [3]:
knapsack_capacity = 6

In [4]:
%%latex 
Binary Knapsack problem with uncertainty in the weight of items. In this problem we assume that the weight of items are known
only after we made our decisions.
$$\max \sum\limits_{s\in S}p_s\sum\limits_{i = 1}^n c_i x_i$$
$$\sum\limits_{i = 1}^n  w^s_i x_i \leq b,\; \forall s \in S$$
$$x_i\in \{0, 1\}\;\forall i=1,\;...,n$$

<IPython.core.display.Latex object>

In [5]:
# generating scenarios:
number_of_scenarios = 128
scenarios = normal(df['weight_mean'], df['weight_std'], [number_of_scenarios, len(df)])

In [6]:
# print average of each parameter value over all scenarios
print([np.average(scenarios[:,0]),np.average(scenarios[:,1]),np.average(scenarios[:,2])])

[4.007956268677221, 2.8101288067527816, 2.186436795633365]


In [7]:
def init_weight_param(model, i, j):
    return scenarios[i, j]

In [8]:
def init_value_param(model, i):
    return df.loc[i, 'value']

In [9]:
def weight_limit_rule(m, j):
    return sum(m.w[j, i] * m.x[i] for i in range(len(df))) <= knapsack_capacity

In [10]:
model = pyo.ConcreteModel()

In [11]:
# instance definition
model.ins_items = pyo.RangeSet(0, len(df)-1)
model.ins_scenarios = pyo.RangeSet(0, number_of_scenarios-1)

In [12]:
# variable definition
model.x = pyo.Var(model.ins_items, domain=pyo.Binary)

In [13]:
# parameter defnition
model.w = pyo.Param(model.ins_scenarios, model.ins_items,
                    initialize=init_weight_param, within=pyo.PositiveReals)
model.v = pyo.Param(model.ins_items, initialize=init_value_param, within=pyo.PositiveReals)

In [14]:
# objective function definition
model.obj = pyo.Objective(expr = sum((1/number_of_scenarios) * sum(model.v[i] * model.x[i] for i in model.ins_items)
                          for _ in model.ins_scenarios), sense=pyo.maximize)

In [15]:
# constraints definition
model.weight_limit_constraints = pyo.Constraint(model.ins_scenarios, rule=weight_limit_rule)

In [16]:
# solving the model
opt = pyo.SolverFactory('cplex')
result = opt.solve(model)

In [17]:
model.display()

Model unknown

  Variables:
    x : Size=3, Index=ins_items
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :   1.0 :     1 : False : False : Binary
          1 :     0 :   0.0 :     1 : False : False : Binary
          2 :     0 :   0.0 :     1 : False : False : Binary

  Objectives:
    obj : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 100.0

  Constraints:
    weight_limit_constraints : Size=128
        Key : Lower : Body               : Upper
          0 :  None :   4.11803477819225 :   6.0
          1 :  None : 3.8918043985366153 :   6.0
          2 :  None :  4.017606917568212 :   6.0
          3 :  None :  3.841097847674472 :   6.0
          4 :  None : 3.9162796798921833 :   6.0
          5 :  None :  4.091632988196975 :   6.0
          6 :  None :  4.099727877533985 :   6.0
          7 :  None :  3.964126573051243 :   6.0
          8 :  None :  4.116533809317629 :   6.0
          9 :  None : 3.954207683